<a href="https://colab.research.google.com/github/sun-in-universe/blue_dot/blob/main/IMDB_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%84%B8%ED%8A%B8%EC%97%90_RNN_%EC%A0%81%EC%9A%A9%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기초 RNN

1. 훈련 세트와 검증세트 준비하기
2. 샘플 길이 맞추고 원-핫 인코딩 하기
3. SImpleRNN 클래스로 RNN 모델 만들기
4. RNN 모델 훈련하기
5. 임베딩층으로 RNN 모델 훈련하기

In [1]:
#매개변수 1. skip_top : 건너뛸 단어 개수 지정 (a, the, is 등)
#매개변수 2. num_words : 훈련에 사용할 단어 개수 지정

import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import imdb

(x_train_all, y_train_all), (x_test, y_test) = imdb.load_data(skip_top=20, num_words=100)

17464789/17464789 [==============================] - 0s 0us/step


In [2]:
#훈련 세트의 샘플 확인하기-영단어를 고유한 정수에 일대일 대응

print(x_train_all[0])

[2, 2, 22, 2, 43, 2, 2, 2, 2, 65, 2, 2, 66, 2, 2, 2, 36, 2, 2, 25, 2, 43, 2, 2, 50, 2, 2, 2, 35, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 39, 2, 2, 2, 2, 2, 2, 38, 2, 2, 2, 2, 50, 2, 2, 2, 2, 2, 2, 22, 2, 2, 2, 2, 2, 22, 71, 87, 2, 2, 43, 2, 38, 76, 2, 2, 2, 2, 22, 2, 2, 2, 2, 2, 2, 2, 2, 2, 62, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 66, 2, 33, 2, 2, 2, 2, 38, 2, 2, 25, 2, 51, 36, 2, 48, 25, 2, 33, 2, 22, 2, 2, 28, 77, 52, 2, 2, 2, 2, 82, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 36, 71, 43, 2, 2, 26, 2, 2, 46, 2, 2, 2, 2, 2, 2, 88, 2, 2, 2, 2, 98, 32, 2, 56, 26, 2, 2, 2, 2, 2, 2, 2, 22, 21, 2, 2, 26, 2, 2, 2, 30, 2, 2, 51, 36, 28, 2, 92, 25, 2, 2, 2, 65, 2, 38, 2, 88, 2, 2, 2, 2, 2, 2, 2, 2, 32, 2, 2, 2, 2, 2, 32]


In [ ]:
#훈련 세트에서 0, 1, 2 제거하기 -> 0=패딩, 1=글의 시작, 2=어휘사전에 없는 단어
#패딩은 입력 시퀀스의 길이를 맞추기 위해, 길이가 짧은 시퀀스에 임의의 값을 추가하는 작업을 의미합니다. 이를 통해 모든 입력 데이터가 동일한 길이를 갖게 되어 모델에 쉽게 입력할 수 있습니다.

for i in range(len(x_train_all)):
  x_train_all[i] = [w for w in x_train_all[i] if w > 2]

  print(x_train_all[0])

In [4]:
# 어휘사전 내려받기
# 훈련세트 -> 영단어 변경위해서는 어휘사전 필요 : get_word_index() 사용

word_to_index = imdb.get_word_index()
word_to_index['movie']

1641221/1641221 [==============================] - 0s 0us/step


17

In [5]:
#총 88584만큼의 영단어
len(word_to_index.keys())

88584

In [47]:
index_to_word = {word: index for index, word in word_to_index.items()}

for w in x_train_all[0]:
  print(index_to_word[w - 3], end=' ')

film just story really they you just there an from so there film film were great just so much film would really at so you what they if you at film have been good also they were just are out because them all up are film but are be what they have don't you story so because all all 

In [48]:
# 훈련세트에 있는 정수는 3이상부터 영단어를 의미-> index_to_word에서 0, 1, 2를 지워줘야 한다.

# index_to_word = {word_to_index[k] for k in word_to_index}

# for w in x_train_all[0]:
#   print(index_to_word[w - 3], end=' ')

In [19]:
# 훈련세트의 입력데이터는 numpy배열이 아님 !

print(len(x_train_all[0]), len(x_train_all[1]))

59 32


In [20]:
# 타깃 데이터는 0(부정), 1(긍정)으로 나타남

print(y_train_all[:10])

[1 0 0 1 0 0 1 0 1 0]


In [21]:
# 25000개 훈련 세트 중 5000개만 분리하여 검증 세트로 사용
# permutation 함수 = 주어진 숫자의 모든 순열 만들기 1 2 3, 2, 1, 3 등등... -> 데이터 섞기

np.random.seed(42)
random_index = np.random.permutation(25000)

x_train = x_train_all[random_index[:20000]]
y_train = x_train_all[random_index[:20000]]

x_val = x_train_all[random_index[20000:]]
y_val = x_train_all[random_index[20000:]]

In [30]:
# 샘플 길이 맞추기 -> pad_sequence() 함수 사용

from tensorflow.keras.preprocessing import sequence

maxlen = 100
x_train_seq = sequence.pad_sequences(x_train, maxlen=maxlen)
x_val_seq = sequence.pad_sequences(x_val, maxlen=maxlen)

# 각 샘플 길이가 100으로 변경 확인

print(x_train_seq.shape, x_val_seq.shape)

# 첫번째 샘플의 왼쪽이 0으로 채워짐

print(x_train_seq[0])

(20000, 100) (5000, 100)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 35 40 27 28 40 22 83 31 85 45
 24 23 31 70 31 76 30 98 32 22 28 51 75 56 30 33 97 53 38 46 53 74 31 35
 23 34 22 58]


<RNN에서 원핫 인코딩 하는 이유>

RNN(Recurrent Neural Network)은 순차적인 데이터를 다루기 위해 사용되는 딥 러닝 모델입니다. RNN은 입력 데이터의 순서를 고려하여 처리하는데, 이러한 특성 때문에 입력 데이터를 적절한 형태로 변환해야 합니다. 원핫 인코딩은 RNN에서 입력 데이터를 처리하기 위한 중요한 전처리 기법 중 하나입니다.

원핫 인코딩은 카테고리 형태의 데이터를 컴퓨터가 이해하기 쉽도록 변환하는 방법 중 하나입니다. 특히 텍스트 데이터와 같이 토큰들이 특정 범주를 나타내는 경우에 많이 사용됩니다. 원핫 인코딩은 다음과 같은 과정을 거칩니다:

데이터에 있는 모든 유니크한 토큰들에 고유한 정수 인덱스를 부여합니다.
각 토큰에 해당하는 인덱스 위치에 1을 표시하고, 나머지 인덱스 위치에는 0을 표시하여 토큰을 벡터 형태로 표현합니다.
예를 들어, 단어 "apple"을 원핫 인코딩으로 표현하면 [0, 0, 0, 1, 0]과 같이 됩니다. 여기서 [0, 0, 0, 1, 0]은 5차원 벡터로, 첫 번째 인덱스가 0이고 다른 인덱스들이 0이기 때문에 "apple"이 해당하는 인덱스 위치에만 1을 가지고 나머지는 0으로 표현됩니다.

RNN에서 훈련 데이터를 원핫 인코딩하는 이유는 다음과 같습니다:

이산적인 텍스트 데이터 표현: 텍스트 데이터는 이산적인 토큰들로 구성되어 있습니다. 원핫 인코딩은 이러한 이산적인 데이터를 컴퓨터가 처리하기 쉬운 연속적인 숫자로 변환하여 입력 데이터를 표현합니다.

쉬운 연산: 원핫 인코딩된 벡터는 각 토큰이 고유한 인덱스 위치에만 1을 가지고 나머지는 0으로 표현되므로, 벡터 간의 내적과 덧셈 등의 연산이 간단합니다. 이러한 연산은 뉴럴 네트워크의 학습과 추론에서 매우 효율적으로 이루어질 수 있습니다.

단어 간 유사성 비교: 원핫 인코딩은 단어 간 유사성을 계산하는데 사용할 수 있습니다. 유사한 단어들은 벡터 공간에서 더 가깝게 위치하게 되므로 단어 간 유사성을 계산하기가 용이합니다.

단어를 원핫 인코딩하여 RNN에 입력으로 사용하면, 모델은 각 토큰을 독립적으로 처리하는 것이 아니라 시퀀스 상의 순서를 고려하여 문맥을 파악하게 됩니다. 이는 자연어 처리(NLP)와 같은 순차적인 데이터를 다루는데 매우 중요한 특성입니다.

In [31]:
from tensorflow.keras.utils import to_categorical

x_train_onehot = to_categorical(x_train_seq)
x_val_onehot = to_categorical(x_val_seq)

print(x_train_onehot.shape)
print(x_train_onehot.nbytes)

(20000, 100, 100)
800000000


In [32]:
# simple rnn 클래스로 rnn 모델 만들기

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [33]:
# SimpleRNN 매개변수 1 = 은닉층 개수 지정, 매개변수 2 = input_shape
# Dense 매개변수 1 = 유닛개수, 매개변수 2 = 활성화 함수

model = Sequential()

model.add(SimpleRNN(32, input_shape=(100, 100)))
model.add(Dense(1, activation='sigmoid'))

In [34]:
model.summary()

#simple_rnn 계층 = 4256개의 파라미터


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 32)                4256      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 4,289
Trainable params: 4,289
Non-trainable params: 0
_________________________________________________________________


In [39]:
print(type(x_train_onehot))
print(type(y_train))
print(type(x_val_onehot))
print(type(y_val))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [41]:
print(x_train_onehot.shape)
print(x_val_onehot.shape)

(20000, 100, 100)
(5000, 100, 100)


In [42]:
#  RNN 모델 훈련하기

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(x_train_onehot, y_train, epochs=20, batch_size=32, validation_data=(x_val_onehot, y_val))

ValueError: ignored

In [ ]:
# 손실률은 커지고, accuracy와 val_acuuracy는 간의 격차가 커짐 -> 과대적합 가능성

import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

## 임베딩층으로 RNN 학습시키기

임베딩층은 자연어 처리(Natural Language Processing, NLP)에서 텍스트 데이터를 처리하기 위해 사용되는 중요한 요소 중 하나입니다. 임베딩층은 단어나 문장을 밀집된(dense) 공간으로 매핑하는 과정을 수행합니다. 이렇게 밀집된 표현은 희소(sparse)한 원핫 인코딩과는 달리 의미적 유사성을 고려하여 효율적으로 표현됩니다.

임베딩층은 과대적합 문제를 직접적으로 해결하는 기법은 아닙니다. 과대적합 문제를 해결하기 위해서는 모델의 복잡도를 조정하거나 정규화 기법 등을 사용해야 합니다. 하지만 임베딩층을 사용하면 과대적합 문제를 완전히 해소하지는 못하지만, 과대적합에 대한 영향을 줄이고 모델의 성능을 개선하는데 도움이 됩니다.

임베딩층이 과대적합 문제를 완전히 해소하지 못하는 이유는 임베딩층 자체가 모델에 파라미터로서 존재하기 때문입니다. 따라서 임베딩층은 모델이 훈련 데이터에 과도하게 적합되는 경우에도 영향을 미칩니다. 하지만 임베딩층은 단어들의 의미적 유사성을 반영하여 단어 간의 관계를 더 잘 이해하도록 도와주기 때문에 훈련 데이터에 대해 더 일반화된 표현을 학습할 수 있습니다.

In [50]:
#임베딩층으로 RNN 모델 훈련하기 => 단어 사이의 관계를 학습시킬 수 있음

from tensorflow.keras.layers import Embedding

(x_train_all, y_train_all), (x_test, y_test) = imdb.load_data(skip_top=20, num_words=1000)

for i in range(len(x_train_all)):
  x_train_all[i] = [w for w in x_train_all[i] if w > 2]

x_train = x_train_all[random_index[:20000]]
y_train = x_train_all[random_index[:20000]]

x_val = x_train_all[random_index[20000:]]
y_val = x_train_all[random_index[20000:]]

In [51]:
maxlen = 100
x_train_seq = sequence.pad_sequences(x_train, maxlen=maxlen)
x_val_seq = sequence.pad_sequences(x_val, maxlen=maxlen)

In [52]:
model_ebd = Sequential()

model_ebd.add(Embedding(1000, 32))
model_ebd.add(SimpleRNN(8))
model_ebd.add(Dense(1, activation='sigmoid'))

model_ebd.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 8)                 328       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 32,337
Trainable params: 32,337
Non-trainable params: 0
_________________________________________________________________


In [53]:
model_ebd.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

history = model_ebd.fit(x_train_seq, y_train, epochs=10, batch_size=32, validation_data=(x_val_seq, y_val))

ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [ ]:
loss, accuracy = model_ebd.evaluate(x_val_seq, y_val, verbose=0)
print(loss, accuracy)